# Projet de traitement de données massive

## Partie 1

### Collecte de données

#### Extract informations

Cloner le dataset depuis git

In [1]:
import git
# Check out via HTTPS
#git.Repo.clone_from('https://github.com/Lseig/Images_TDM.git', 'Images_git')

Extraction des informations des images

In [1]:
import PIL.Image
import os
import json
from sklearn.cluster import KMeans
import numpy
from colorthief import ColorThief
import time
import git

from scipy.spatial import KDTree
from webcolors import hex_to_rgb
from webcolors import CSS3_HEX_TO_NAMES


class image_analysis:
    def __init__(self, directory):
        self.extract_metadata(directory)

    def orientation(self, heigh, width):
        if heigh > width:
            return "Portrait"
        elif width > heigh:
            return "Paysage"
        else:
            return "Carre"

    def convert_rgb_to_names(self, rgb_tuple):
        # a dictionary of all the hex and their respective names in css3
        css3_db = CSS3_HEX_TO_NAMES
        names = []
        rgb_values = []
        for color_hex, color_name in css3_db.items():
            # Ajoute à la liste names les noms des couleurs
            names.append(color_name)
            rgb_values.append(hex_to_rgb(color_hex))

        kdt_db = KDTree(rgb_values)
        distance, index = kdt_db.query(rgb_tuple)
        return f'{names[index]}'

    def extract_metadata(self, directory_name):
        directory = os.fsencode(directory_name)
        dataDict = {}
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            img = PIL.Image.open(directory_name+"/"+filename)
            exif_data = img._getexif()
            color_thief = ColorThief(directory_name+"/"+filename)
            tabcolor = color_thief.get_palette(color_count=5)
            dataDict[filename] = {
                "Height": img.height,
                "Width": img.width,
                "Exif": str(exif_data),
                "Couleur dominante 1": self.convert_rgb_to_names(tabcolor[0]),
                "Couleur dominante 2": self.convert_rgb_to_names(tabcolor[1]),
                "Couleur dominante 3": self.convert_rgb_to_names(tabcolor[2]),
                "Couleur dominante 4": self.convert_rgb_to_names(tabcolor[3]),
                "Couleur dominante 5": self.convert_rgb_to_names(tabcolor[4]),
                "Orientation": self.orientation(img.height, img.width),
            }

        with open('json_data.json', 'w') as outfile:
            json.dump(dataDict, outfile)


In [ ]:
class algo_classification:
    def __init__(self,training_file , image_folder):
        self.training_file = training_file
        t = image_analysis(image_folder)
        self.concatenateLikesAnalysis()
    
    def concatenateLikesAnalysis(self):
        like = []
        dislike = []
        analysis_file = open('json_data.json', 'r')
        aDict = json.load(analysis_file)
        analysis_file.close()

        training_fileA = open(self.training_file)
        likeDict = json.load(training_fileA)
        training_fileA.close()

        for x in aDict:
            for y in likeDict:
                if x == y:   
                    if likeDict[y]["like"] == "y":
                        like.append(aDict[x])
                    elif likeDict[y]["like"] == "n":
                        dislike.append(aDict[x])   

test = algo_classification("training_file.json", "image_one")


In [ ]:
def generate_training_file():
    like_dict = {}
    
    directory = os.fsencode("image_like")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        
        like = input("Entrer Y si vous aimez l'image"+ filename + " ou N si vous ne l'aimez pas ou U si vous n'avez pas d'avis")
        like_dict[filename] = {"like": like}
    with open('training_file.json', 'w') as outfile:
        json.dump(like_dict, outfile)


generate_training_file()